In [ ]:
!pip install ultralytics
import  ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO
import torch
import torch.nn as nn
from ultralytics.nn.modules import C2f, SPPF, Conv
import time
from datetime import datetime

# YOLOv8 modelini özelleştirmek için bir sınıf tanımlıyoruz
class CustomYOLOv8(YOLO):
    def __init__(self, model='yolov8n.pt'):
        super().__init__(model)

        # Derinlik artırımı için kanal sayıları (YOLOv8n için uygun boyutlar)
        c2f_1_channels = 256
        c2f_2_channels = 512
        sppf_channels = 512
        head_channels = 256

        # C2f modülü: Derinliği artırarak modelin daha fazla özellik öğrenmesini sağlar.
        # n=2 ile 2 adet tekrar bloğu kullanılır, bu da modelin karmaşık örüntüleri daha iyi yakalamasına yardımcı olur.
        self.model.model[4] = C2f(c2f_1_channels, c2f_1_channels, n=2, shortcut=True)
        self.model.model[6] = C2f(c2f_2_channels, c2f_2_channels, n=2, shortcut=True)

        # SPPF (Spatial Pyramid Pooling Fast): Farklı ölçeklerdeki bilgiyi birleştirerek modele çoklu bağlam (context) kazandırır.
        # Burada `k=5` kullanılarak daha büyük çekirdek ile daha geniş bağlam öğrenimi sağlanır.
        self.model.model[9] = SPPF(sppf_channels, sppf_channels, k=5)

        # Head kısmındaki C2f katmanı: Çıkış katmanından önceki özellikleri daha iyi işler.
        self.model.model[15] = C2f(head_channels, head_channels, n=2, shortcut=True)

        # Özelleştirilmiş attention modülü: Hem uzamsal (spatial) hem de kanal (channel) bazlı dikkat mekanizması uygular.
        # Bu modül, önemli bölgeleri ön plana çıkararak modelin odaklanmasını sağlar.
        self.model.model.insert(8, AttentionModule(512))

    def train(self, **kwargs):
        # Eğitim parametrelerini özelleştiriyoruz
        custom_args = {
            'epochs': 100,                # 100 epoch boyunca eğitim yapılacak
            'batch': 16,                  # Her seferde 16 örnek işlenecek
            'workers': 8,                 # Veri yüklemek için 8 CPU iş parçacığı kullanılacak
            'device': 'cpu',              # Eğitim CPU’da yapılacak (GPU varsa 'cuda' önerilir)

            # Optimizasyon parametreleri
            'optimizer': 'SGD',           # SGD (Stochastic Gradient Descent) daha kararlı ve kontrollü öğrenme sağlar
            'lr0': 0.01,                  # Başlangıç öğrenme oranı – küçük veri setlerinde daha yüksek olabilir
            'lrf': 0.01,                  # Son öğrenme oranı (final learning rate), lr0'dan bu değere kadar lineer azalır
            'momentum': 0.937,            # Ağırlıkların güncellenme hızını kontrol eder – geçmiş bilgilere önem verir
            'weight_decay': 0.0005,       # Aşırı öğrenmeyi (overfitting) önlemek için ağırlıkları cezalandırır

            # Isınma (warmup) ayarları – başlangıçta daha yavaş ve güvenli öğrenme
            'warmup_epochs': 3.0,
            'warmup_momentum': 0.8,
            'warmup_bias_lr': 0.1,

            # Kayıp fonksiyonu ağırlıkları (Loss components)
            'box': 7.5,                   # Kutu tahmini hatasına verilen önem
            'cls': 0.5,                   # Sınıflandırma hatasına verilen önem
            'dfl': 1.5,                   # Distribution Focal Loss – kutu tahmininde daha hassas sonuçlar sağlar

            # Görüntü veri artırma (augmentation) ayarları
            'hsv_h': 0.015,               # Renk tonunda küçük değişiklik
            'hsv_s': 0.7,                 # Doygunluk değişimi – daha geniş varyasyon sağlar
            'hsv_v': 0.4,                 # Parlaklık değişimi – ışık koşullarını simüle eder
            'degrees': 0.0,               # Dönme yok (cilt verisi için gerek görülmemiş)
            'translate': 0.1,             # Küçük kaydırmalar uygulanır
            'scale': 0.5,                 # Ölçekleme uygulanır – farklı yüz boyutları için yararlı
            'shear': 0.0,                 # Kesme işlemi yok
            'perspective': 0.0,           # Perspektif bozulma yok
            'flipud': 0.0,                # Yukarı-aşağı çevirme kapalı (cilt verilerinde anlamlı değil)
            'fliplr': 0.5,                # Yatay çevirme – yüz simetrisi sayesinde anlamlı
            'mosaic': 1.0,                # Mozaik birleştirme – veri çeşitliliğini artırır
            'mixup': 0.0,                 # Mixup kapalı – cilt verilerinde karışıklığa yol açabilir
            'copy_paste': 0.0,            # Kopyala-yapıştır augmentation kapalı
        }

        # Dışarıdan gelen parametrelerle güncelle
        custom_args.update(kwargs)
        return super().train(**custom_args)

# Dikkat modülü – hem kanal hem de uzamsal dikkat uygular
class AttentionModule(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.spatial_attention = nn.Sequential(
            nn.Conv2d(channels, channels//8, 1),
            nn.SiLU(),
            nn.Conv2d(channels//8, channels, 1),
            nn.Sigmoid()
        )

        self.channel_attention = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels//8, 1),
            nn.SiLU(),
            nn.Conv2d(channels//8, channels, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        sa = self.spatial_attention(x)  # Uzamsal dikkat maskesi
        ca = self.channel_attention(x)  # Kanal dikkat maskesi
        return x * sa * ca  # Giriş ile çarpılarak önemli bölgeler vurgulanır

# Belirli bir cilt durumu için özel modeli oluşturur ve eğitir
def create_custom_model(data_yaml, model_name):
    print(f"\n{'='*50}")
    print(f"Model eğitimi başlıyor: {model_name}")
    print(f"Başlangıç zamanı: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"{'='*50}\n")

    # Özelleştirilmiş YOLOv8 modeli oluşturuluyor
    model = CustomYOLOv8('yolov8n.pt')

    # Eğitim başlatılıyor
    model.train(
        data=data_yaml,
        name=f'yolov8n_{model_name}_detection_custom',
    )

    print(f"\n{'='*50}")
    print(f"Model eğitimi tamamlandı: {model_name}")
    print(f"Bitiş zamanı: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"{'='*50}\n")

    return model

# Tüm cilt koşulları için modelleri sırayla eğitir
def train_all_models():
    data_paths = {
        'wrinkles': '/content/drive/MyDrive/Skin_Conditions/data/wrinkles.yaml',
        'eyebags': '/content/drive/MyDrive/Skin_Conditions/data/eyebags.yaml',
        'acne': '/content/drive/MyDrive/Skin_Conditions/data/acne.yaml',
        'redness': '/content/drive/MyDrive/Skin_Conditions/data/redness.yaml'
    }
    
    models = {}
    total_start_time = time.time()
    
    print("\nEğitim süreci başlıyor...")
    print(f"Toplam eğitilecek model sayısı: {len(data_paths)}")
    
    for i, (condition, yaml_path) in enumerate(data_paths.items(), 1):
        print(f"\nModel {i}/{len(data_paths)}: {condition}")
        models[condition] = create_custom_model(yaml_path, condition)
        
        # Eğitilen modeli kaydet
        save_path = f'models/{condition}_model.pt'
        models[condition].save(save_path)
        print(f"Model kaydedildi: {save_path}")
    
    total_time = time.time() - total_start_time
    print(f"\n{'='*50}")
    print("Tüm eğitimler tamamlandı!")
    print(f"Toplam eğitim süresi: {total_time/3600:.2f} saat")
    print(f"{'='*50}\n")
    
    return models

# Ana çalıştırıcı – script doğrudan çalıştırıldığında tüm modeller eğitilir
if __name__ == "__main__":
    trained_models = train_all_models()
